In [1]:
from pyknow import *
import random

NERD = True

In [2]:
class WinTotals(Fact):
    pass


class Results(Fact):
    pass


class ValidAnswer(Fact):
    pass


class Action(Fact):
    pass


class HumanChoice(Fact):
    pass


class ComputerChoice(Fact):
    pass


class RockPaperScissors(KnowledgeEngine):
    def __init__(self, *args, is_nerd=False, **kwargs):
        self.valid_answers = dict()
        self.is_nerd = is_nerd
        super().__init__(*args, **kwargs)

    def yes_or_no(self, question):
        return input(question).upper().startswith('Y')
    
    @DefFacts()
    def game_rules(self):
        """Declare game rules and valid input keys for the user."""
        
        yield Results(winner='rock', loser='scissors', why='Rock smashes scissors')
        yield Results(winner='paper', loser='rock', why='Paper covers rock')
        yield Results(winner='scissors', loser='paper', why='Scissors cut paper')
        yield ValidAnswer(answer='rock', key='r')
        yield ValidAnswer(answer='paper', key='p')
        yield ValidAnswer(answer='scissors', key='s')

        if self.is_nerd:
            yield Results(winner='rock', loser='lizard', why='Rock crushes lizard')
            yield Results(winner='spock', loser='rock', why='Spock vaporizes rock')
            yield Results(winner='spock', loser='scissors', why='Spock smashes scissors')
            yield Results(winner='paper', loser='spock', why='Paper disproves Spock')
            yield Results(winner='scissors', loser='lizard', why='Scissors decapitates lizard')
            yield Results(winner='lizard', loser='paper', why='Lizard eats paper')
            yield Results(winner='lizard', loser='spock', why='Lizard poisons Spock')
            yield ValidAnswer(answer='spock', key='k')
            yield ValidAnswer(answer='lizard', key='l')
    
    @Rule()
    def startup(self):
        print("Lets play a game!")
        print("You choose rock, paper, or scissors,")
        print("and I'll do the same.")
        self.declare(WinTotals(human=0, computer=0, ties=0))
        self.declare(Action('get-human-move'))
    
    @Rule(NOT(Action()),
          ValidAnswer(answer='answer' << W(),
                      key='key' << W()))
    def store_valid_answers(self, answer, key):
        self.valid_answers[key] = answer

    #
    # HUMAN MOVE RULES
    #
    @Rule(Action('get-human-move'))
    def get_human_move(self):
        question = ", ".join(
            "{name} ({key})".format(
                name=a[1].title(), key=a[0].upper())
            for a in self.valid_answers.items()) + '? '
        res = input(question).lower()
        self.declare(HumanChoice(res))
    
    @Rule('f1' << HumanChoice('choice' << W()),
          ValidAnswer(answer='answer' << W(),
                      key='choice' << W()),
          'f2' << Action(L('get-human-move')))
    def good_human_move(self, f1, f2, choice, answer):
        self.retract(f1)
        self.retract(f2)
        self.declare(HumanChoice(answer))
        self.declare(Action('get-computer-move'))
    
    @Rule('f1' << HumanChoice('choice' << W()),
          ~ValidAnswer(key='choice' << W()),
          'f2' << Action('get-human-move'))
    def bad_human_move(self, f1, f2, choice):
        print("Sorry %s is not a valid answer" % choice)
        self.retract(f1)
        self.retract(f2)
        self.declare(Action('get-human-move'))
    
    #
    # COMPUTER MOVE RULES
    #
    @Rule('f1' << Action('get-computer-move'))
    def get_computer_move(self, f1):
        choice = random.choice(list(self.valid_answers.values()))
        self.retract(f1)
        self.declare(ComputerChoice(choice))
        self.declare(Action('determine-results'))

    #
    # WIN DETERMINATION RULES
    #
    @Rule('f1' << Action('determine-results'),
          'f2' << ComputerChoice('cc' << W()),
          'f3' << HumanChoice('hc' << W()),
          'w' << WinTotals(computer='cw' << W()),
          Results(winner='cc' << W(),
                  loser='hc' << W(),
                  why='explanation' << W()))
    def computer_wins(self, f1, f2, f3, w, cc, hc, cw, explanation):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, computer=cw + 1)
        print("Computer wins!", explanation)
        self.declare(Action('determine-play-again'))
        
    @Rule('f1' << Action('determine-results'),
          'f2' << ComputerChoice('cc' << W()),
          'f3' << HumanChoice('hc' << W()),
          'w' << WinTotals(human='hw' << W()),
          Results(winner='hc' << W(),
                  loser='cc' << W(),
                  why='explanation' << W()))
    def humans_wins(self, f1, f2, f3, w, cc, hc, hw, explanation):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, human=hw + 1)
        print("You win!", explanation)
        self.declare(Action('determine-play-again'))
        
    @Rule('f1' << Action('determine-results'),
          'f2' << ComputerChoice('cc' << W()),
          'f3' << HumanChoice('cc' << W()),
          'w' << WinTotals(ties='nt' << W()))
    def tie(self, f1, f2, f3, w, cc, nt):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, ties=nt + 1)
        print("Tie! Ha-ha!")
        self.declare(Action('determine-play-again'))
    
    #
    # PLAY AGAIN RULE
    #
    @Rule('f1' << Action('determine-play-again'),
          WinTotals(computer='ct' << W(),
                    human='ht' << W(),
                    ties='tt' << W()))
    def play_again(self, f1, ct, ht, tt):
        self.retract(f1)
        if not self.yes_or_no("Play again?"):
            print("You won", ht, "game(s).")
            print("Computer won", ct, "game(s).")
            print("We tied", tt, "game(s).")
            self.halt()
        else:
            self.declare(Action('get-human-move'))

In [3]:
rps = RockPaperScissors()

In [4]:
rps.reset()
rps.run()

Lets play a game!
You choose rock, paper, or scissors,
and I'll do the same.
Scissors (S), Rock (R), Paper (P)? p
Computer wins! Scissors cut paper
Play again?y
Scissors (S), Rock (R), Paper (P)? r
Tie! Ha-ha!
Play again?y
Scissors (S), Rock (R), Paper (P)? s
Tie! Ha-ha!
Play again?n
You won 0 game(s).
Computer won 1 game(s).
We tied 2 game(s).


In [6]:
rps = RockPaperScissors(is_nerd=True)
rps.reset()
rps.run()

Lets play a game!
You choose rock, paper, or scissors,
and I'll do the same.
Lizard (L), Paper (P), Scissors (S), Rock (R), Spock (K)? k
Computer wins! Paper disproves Spock
Play again?y
Lizard (L), Paper (P), Scissors (S), Rock (R), Spock (K)? k
Computer wins! Paper disproves Spock
Play again?y
Lizard (L), Paper (P), Scissors (S), Rock (R), Spock (K)? k
Computer wins! Lizard poisons Spock
Play again?y
Lizard (L), Paper (P), Scissors (S), Rock (R), Spock (K)? k
Tie! Ha-ha!
Play again?n
You won 0 game(s).
Computer won 3 game(s).
We tied 1 game(s).
